# Getting Started with Snowflake Notebooks 

Welcome to the 2026 EY Data & AI Challenge! 

This notebook will get you up and running with Snowflake Notebooks. By the end, you'll see how Snowflake can help you leverage a powerful cloud-computing environment to gather satellite imagery data from an external source and conduct geospatial analyses using that data. Once you successfully run this notebook, you will understand the steps required to use Python notebooks for the data challenge. Dive in and have fun !!!

## Confirm setup of External Access Integration (EAI)

Before running this notebook, you will need to run a SQL script to create an [External Access Integration](https://docs.snowflake.com/en/developer-guide/external-network-access/external-network-access-overview). 

This script creates external network access to install PyPI packages and access Planetary Computer API endpoints. 

The setup script can be accessed directly within Snowflake using this [deeplink](https://app.snowflake.com/templates?template=setup_account_data_challenge_template).

The SQL script is also located in the Snowflake-Labs repo on [Github](https://github.com/Snowflake-Labs/sfquickstarts/tree/master/site/sfguides/src/ey-ai-and-data-challenge).

An EAI relies upon network rules that are stored in a database. You can attach many rules to the same policy. This same process can be used to setup other access points for external datasets from Google or Amazon.

Snowflake notebooks can run SQL and Python cells. When you run the next cell, you should see a dialog box where you can choose which External Access Integration you wish to use. Select "DATA_CHALLENGE_EXTERNAL_ACCESS". Click "Create and Connect." It will take 2-3 minutes for the kernel to connect to the container service.

When you see that the kernel status is "Connected", then the SQL cell will run, and "DATA_CHALLENGE_EXTERNAL_ACCESS" will be listed in the output. 

In [ ]:
show integrations;

## Import Python packages
Snowflake Notebooks have 2 runtime options: On Warehouse or On Container. 
When you run Snowflake Notebooks inside Snowflake Workspaces, notebooks run on Container, so you can easily use "pip install" to load python libraries. 

The Container runtime contains Streamlit and many other third-party packages (by default)[https://docs.snowflake.com/en/developer-guide/snowflake-ml/container-runtime-ml#snowflake-runtime-packages]. 

You can add any additional packages from [PyPI- Python Package Index](www.pypi.org) by loading all dependencies in a **requirements.txt** file. You can create the file by running the Python cell below, or manually upload your **requirements.txt** file into the same folder as this notebook using the plus button in the left sidebar. 

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Step 1: Create the requirements.txt file locally
requirements = """
rioxarray==0.17.0
pystac==1.11.0
pystac_client==0.9.0
planetary_computer==1.0.0
odc-stac==0.3.10
shapely==2.1.1
rasterio==1.4.3
tqdm==4.66.5
pandas==2.3.0
xarray==2025.3.1
matplotlib==3.10.3
geopandas==1.1.1
seaborn==0.13.2
scikit-learn==1.5.2
netCDF4==1.7.2
adlfs==2025.8.0
zarr==2.17.2
dask==2024.10.0
xarray[complete]
numcodecs==0.12.1
"""

with open('/tmp/requirements.txt', 'w') as f:
    f.write(requirements)

print("File created locally")

# Step 2: Upload to notebook stage using SQL

session.sql(f"""
    PUT file:///tmp/requirements.txt 
    snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
    AUTO_COMPRESS=FALSE 
    OVERWRITE=TRUE
""").collect()

print("Uploaded to notebook environment! Refresh your browser tab to see in sidebar.")

## File Management
The method above stores files in the notebook environment. These files persist across sessions.

It is also possible to save files from a Notebook to a Snowflake Internal Stage. [Read the docs](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-work-with-files#store-files-in-a-snowflake-stage) for more info on these different options.

To save a file from your python environment into a Snowflake stage, use this syntax:

In [ ]:
session.sql(f"""
    CREATE STAGE IF NOT EXISTS SNOWFLAKE_LEARNING_DB.PUBLIC.MY_STAGE
    DIRECTORY = ( ENABLE = TRUE)
""").collect()

session.file.put('requirements.txt', "@SNOWFLAKE_LEARNING_DB.PUBLIC.MY_STAGE",
    auto_compress = False,
    overwrite = True)
print("Uploaded to Stage!")

In [ ]:
list @SNOWFLAKE_LEARNING_DB.PUBLIC.MY_STAGE;

## Python Package Management

Snowflake Notebooks operate similar to Jupyter notebooks that run in your local environment, with one significant difference in the way storage is handled. 

When you run "pip install" in your local environment, there is a physical folder structure where the packages are stored. In Snowflake, there are 2 storage layers: the persistent storage layer (that you can see in the sidebar to the left) and the container storage layer that resets each time your Notebook Container Service restarts. 

By default, the Notebook Service has an idle timeout of 24 hours, but the Compute Pool (which uses Snowflake Credits) has an idle timeout of 5 minutes. This means that after 5 minutes, you stop using up your Snowflake Credits, but the variables and packages in the kernel will remain active for 24 hours. For more information, read the [Snowflake Docs](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-in-workspaces/notebooks-in-workspaces-compute-setup).

In [ ]:
import time

start_time = time.time()
print("Installing packages to shared directory...")
print("This may take a few minutes...\n")

# Install all packages from requirements.txt to the shared directory
!pip install uv
!uv pip install  -r requirements.txt 

elapsed_time = time.time() - start_time
print(f"\n✓ Package installation complete!")
print(f"  Time elapsed: {elapsed_time:.1f} seconds ({elapsed_time/60:.1f} minutes)")

## Restart Kernel

After installing packages, it is necessary to restart the kernel, to access the new python packages.
Click the "⌄" symbol next to the  "Connect/Connected" button at the top of the editing pane in [Snowflake Workspaces](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-in-workspaces/notebooks-in-workspaces-overview).

## Load Python Dependencies


In [ ]:
# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load

import dask
import dask.array as da

# Load the satellite data
First, we define our area of interest using latitude and longitude coordinates of the centroid. Then we define the size of the surrounding bounding box (in degrees). Finally, we define the time window.

In [2]:
# Sample region in South Africa
# Contains Water Quality Sample Site #184 and #186 on the Wilge River
lat_long = (-27.2923, 28.5365) # Lat-Lon centroid location
box_size_deg = 0.15 # Surrounding box in degrees

In [3]:
# Calculate the Lat-Lon bounding box region
min_lon = lat_long[1]-box_size_deg/2
min_lat = lat_long[0]-box_size_deg/2
max_lon = lat_long[1]+box_size_deg/2
max_lat = lat_long[0]+box_size_deg/2
bounds = (min_lon, min_lat, max_lon, max_lat)

In [4]:
# Define the time window
time_window="2015-01-01/2015-05-01"

Using the `pystac_client` we can search the Microsoft Planetary Computer's STAC catalog for items matching our query parameters. The result is the number of scenes matching our search criteria that touch our area of interest. Some of these may be partial scenes and may contain clouds. We have included both Landsat-7 and Landsat-8 data and filtered for low-cloud (<10%) scenes.

In [5]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = stac.search(
    collections=["landsat-c2-l2"], 
    bbox=bounds, 
    datetime=time_window,
    query={"platform": {"in": ["landsat-7", "landsat-8"]}, "eo:cloud_cover": {"lt": 10}},
)
items = list(search.get_all_items())
print('This is the number of scenes that touch our region:',len(items))

Next, we'll load the data into an [xarray](https://xarray.pydata.org/en/stable/) DataArray using the Open Data Cube (ODC) STAC [odc-stac](https://odc-stac.readthedocs.io/en/latest/index.html) library. The ODC [odc](https://www.opendatacube.org) is an open source geospatial data management and analysis software project that is used globally for many projects (e.g., Digital Earth Africa). The ODC-STAC code will load the selected items from the catalog search, select the desired spectral bands, including the "qa_pixel" cloud filtering band, reproject into Lat-Lon coordinates (EPSG:4326) at 30-meters resolution (typical of Landsat pixel resolution), and clip the region to the spatial bounding box. 

In [6]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 30  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 

In [7]:
xx = stac_load(
    items,
    bands=["red", "green", "blue", "nir08", "swir16", "swir22", "qa_pixel"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    patch_url=pc.sign,
    bbox=bounds
)

In [8]:
# Apply scaling and offsets for Landsat Collection-2 (reference below) to the spectral bands ONLY
# https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2
xx['red'] = (xx['red']*0.0000275)-0.2
xx['green'] = (xx['green']*0.0000275)-0.2
xx['blue'] = (xx['blue']*0.0000275)-0.2
xx['nir08'] = (xx['nir08']*0.0000275)-0.2
xx['swir16'] = (xx['swir16']*0.0000275)-0.2
xx['swir22'] = (xx['swir22']*0.0000275)-0.2

In [9]:
# View the dimensions of our XARRAY and the variables
display(xx)

## View RGB (real color) images from the time series

Data is available from Landsat-7 from April 1999. Data is available for Landsat-8 from April-2013 to now. A review of the available data shows that the Landdsat-7 scenes have a "banding" issue with the sensor that adds lines of "no data" to each scene. These can be removed by pixel screening but unfortunately some data is lost. So, for the selected time window and missions there are typically views of our region every 8 days. We will pick a scene with some clouds to demonstrate filtering to find the clear pixels.

In [ ]:
# Test that packages work together
import xarray as xr
import dask
import dask.array as da
import zarr
import numpy as np

print("Package Versions:")
print(f"  xarray: {xr.__version__}")
print(f"  dask: {dask.__version__}")
print(f"  zarr: {zarr.__version__}")

# Test dask chunk manager
print("\nTesting dask chunk manager...")
try:
    test_array = da.ones((10, 10), chunks=(5, 5))
    test_xr = xr.DataArray(test_array)
    print(f"✓ Dask chunks working: {test_xr.chunks}")
except Exception as e:
    print(f"✗ Error: {e}")

print("\n✓ Ready to use!")

In [10]:
plot_xx = xx[["red","green","blue"]].to_array()
plot_xx.plot.imshow(col='time', col_wrap=4, robust=True, vmin=0, vmax=0.3)
plt.show()

In [35]:
# Select a time slice to view a simple RGB image and the cloud mask
# See the XARRAY dimensions above for the number of time slices (starts at 0)

time_slice = 1

In [36]:
# Plot and RGB Real Color Image for a single date
fig, ax = plt.subplots(figsize=(8, 8))
xx.isel(time=time_slice)[["red", "green", "blue"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=0.3)
ax.set_title("RGB Real Color")
ax.axis('off')
plt.show()

## Apply Cloud Filtering and Masking

Cloud masking for Landsat Collection-2 Level-2 data can be complicated. The **"qa_pixel"**  band is used to assess each pixel and determine its cloud or water classification. The code below is credited to Mauricio Cordiero. See his webpage [HERE](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat-8-9-C2-L2-ScienceProductGuide-v4.pdf) for a more detailed explanation of the process. Also, you can visit the Landsat-8 Collection-2 Level-2 Product Guide [HERE](https://www.usgs.gov/media/files/landsat-8-9-collection-2-level-2-science-product-guide) for more details about the cloud mask values. In the end, we are searching for clouds, cloud shadows, and water to create a mask for any given region. This will allow us to extract the "clear" pixels to assess the vegetation state. 

In [37]:
# To mask the pixels and find clouds or water, it is best to use the bit values of the 16-bit qa_pixel flag
# See the website above for a nice explanation of the process

bit_flags = {
            'fill': 1<<0,
            'dilated_cloud': 1<<1,
            'cirrus': 1<<2, 
            'cloud': 1<<3,
            'shadow': 1<<4, 
            'snow': 1<<5, 
            'clear': 1<<6,
            'water': 1<<7
}

In [38]:
# Create a function that will mask pixels with a given type

def get_mask(mask, flags_list):
    
    # Create the result mask filled with zeros and the same shape as the mask
    final_mask = np.zeros_like(mask)
    
    # Loop through the flags  
    for flag in flags_list:
        
        # get the mask for each flag
        flag_mask = np.bitwise_and(mask, bit_flags[flag])
        
        # add it to the final flag
        final_mask = final_mask | flag_mask
    
    return final_mask > 0

In [39]:
# Pick a single time slice to view a mask with clouds and water
sample_xx = xx.isel(time=time_slice)

In [40]:
# Find the pixels that are no data (fill), clouds, cloud shadows, or water
my_mask = get_mask(sample_xx['qa_pixel'], ['fill', 'dilated_cloud', 'cirrus', 'cloud', 'shadow', 'water'])

In [41]:
# Show only the mask (Yellow) with valid data in Purple
plt.figure(figsize=(7,7))
plt.imshow(my_mask)
plt.title("Cloud / Shadows / Water Mask > YELLOW")
plt.axis('off')
plt.show()

### Prepare a data mask before plotting sample products

In [42]:
# Calculate the mask for the entire xarray (all time slices)
full_mask = get_mask(xx['qa_pixel'], ['fill', 'dilated_cloud', 'cirrus', 'cloud', 'shadow', 'water'])

In [43]:
# Create a "clean" dataset with the mask applied 
cleaned_data = xx.where(~full_mask)

### Normalized Difference Vegetation Index (NDVI)

The **Normalized Difference Vegetation Index (NDVI)** is used to measure the "greenness" of vegetation and has a range of 0.0 to 1.0. Low values (0.0 to 0.25) reflect a lack of vegetation (bare soil, water), middle values (0.25 to 0.6) reflect crops in their growing state, and high values (0.6 to 1.0) reflect crops at their peak vegetation state. The equation uses two spectral bands where: **NDVI = (NIR-Red) / (NIR+Red)**.

In [44]:
# Plot an NDVI image for a single date with few clouds
fig = plt.figure(figsize=(10, 7))
ndvi_image = (cleaned_data.nir08-cleaned_data.red)/(cleaned_data.nir08+cleaned_data.red)
ndvi_image.isel(time=time_slice).plot(vmin=0.0, vmax=0.8, cmap="RdYlGn")
plt.title("NDVI")
plt.axis('off')
plt.show()

### Normalized Difference Moisture Index (NDMI)

The **Normalized Difference Moisture Index (NDVI)** is used to measure the presence of soil moisture stress level of vegetation and has a range of -1.0 to 1.0. Higher values (0.4 to 1.0) indicate high vegetation water content, often seen in healthy, well-irrigated areas. Lower values (0 to 0.2) suggest low water content or potential water stress. Very low values (-1.0 to 0.0) typically correspond to bare soil or areas with little to no vegetation moisture. The equation uses two spectral bands where: **NDMI = (NIR-SWIR16) / (NIR+SWIR16)**.

In [45]:
# Plot an NDMI image for a single date with few clouds
fig = plt.figure(figsize=(10, 7))
ndvi_image = (cleaned_data.nir08-cleaned_data.swir16)/(cleaned_data.nir08+cleaned_data.swir16)
ndvi_image.isel(time=time_slice).plot(vmin=-0.2, vmax=0.7, cmap="jet_r")
plt.title("NDMI")
plt.axis('off')
plt.show()

### SWIR-22 (Shortwave Infrared #2)
The Shortwave Infrared #2 band (SWIR22) has a wavelength range of 2.11 to 2.29 microns. This band is typically used for characterizing vegetation and land cover, with applications in estimating fractional crop residue cover, monitoring crop health, and distinguishing soil from vegetation. It is a critical tool for long-term land use analysis and change detection in agriculture, forestry, and coastal environments. 

In [46]:
# Plot an SWIR-22 image for a single date with few clouds
fig = plt.figure(figsize=(10, 7))
swir22_image = cleaned_data.swir22
swir22_image.isel(time=time_slice).plot(vmin=0.0, vmax=0.3, cmap="Greys")
plt.title("SWIR22")
plt.axis('off')
plt.show()

## Conclusion

Congratulations! You have successfully connected your Snowflake account to an external API, processed a variety of geospatial data, and generated complex visuals.

### We wish you and your team the best of luck in the Data Challenge!